# 3. Extracción de _Features_ Cuello de Botella

Una de las ventajas más grandes de _transfer learnning_ es que nos permite aprovechar la enorme cantidad de conocimiento almacenado en las capas de los modelos pre-entrenados a nuestra disposición. En un mundo donde obtener data es muy costoso tanto en tiempo como en dinero, esta característica es bastante atractiva.

Si pensamos al respecto, esto modelos pre-entrenados pueden servir como excelentes extractores de _features_, si decidimos deshacernos de sus últimas capas (las cuales tienen a ser redes totalmente conectadas usadas para clasificar imágenes en ImageNet).

Con estos _features_ en mano, podemos anexar _cualquier_ modelo de machine learning, incluyendo (más no limitados a) redes neuronales, siempre y cuando llevemos a cabo las debidas transformaciones requeridas por el algoritmo receptor.

Los _features_ resultantes de esta metodología se conocen como __features cuello de botella__.

## Extractores de _Features_

Aunque cualquier modelo pre-entranado puede actuar como un extractor de _features_, en [este](https://github.com/jesus-a-martinez-v/deep-learning-car-detector) proyecto determinamos que el que mejor funciona para este problema es VGG16, por lo que es lógico reutilizarlo para este experimento.

In [1]:
import numpy as np
from glob import glob
import random

In [2]:
from tensorflow.keras.applications.vgg16 import VGG16

base = VGG16(include_top=False)

/usr/lib/python3/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Colocations handled automatically by placer.


Definiremos tres extractores:

  - **Global Average Pooling Extractor**: Para este extractor estamos aplicando la operación GlobalAveragePooling2D a las salidas del modelo pre-entrenado (sin las capas superiores). Una agrupación global promediada calcula, como su nombre lo indica, el promedio de los valores en un _feature map_ o kernel. Por tanto, si tenemos un volumen de _features_ con 32 filtros de 64x64, obtendremos un vector de 32 elementos.
  - **Global Max Pooling Extractor**: Para este extractor estamos aplicando la operación GlobalMaxPooling2D a las salidas del modelo pre-entrenado (sin las capas superiores). Una agrupación global maximizada calcula, como su nombre lo indica, el máximo de los valores en un _feature map_ o kernel. Por tanto, si tenemos un volumen de _features_ con 32 filtros de 64x64, obtendremos un vector de 32 elementos.
  - **Flatten Extractor**: Este es el extractor más simple, puesto que sólo reagrupa las salidas del modelo (sin las capas superiores) en un vector muy largo. Así, por ejemplo, para un volumen de _features_ de 32 filtros de 64x64, el resultado será un vector de 64 * 32 * 32 = 65536 elementos.

In [3]:
from tensorflow.keras.layers import GlobalAveragePooling2D, GlobalMaxPool2D, Flatten
from tensorflow.keras.models import Model

x = base.output
out = GlobalAveragePooling2D()(x)
        
global_average_feature_extractor = Model(inputs=base.input, outputs=out)

In [4]:
x = base.output
out = GlobalMaxPool2D()(x)
        
global_max_feature_extractor = Model(inputs=base.input, outputs=out)

In [5]:
x = base.output
out = Flatten()(x)
        
flatten_feature_extractor = Model(inputs=base.input, outputs=out)

## Transformaciones del Conjunto de Datos

Con estos extractores definidos, podemos usarlos para generar nuevas versiones, transformaciones del conjunto de datos original. Nuestra meta con este projecto es utilizar deep learning para producir _features_ útiles para algoritmos más clásicos, como árboles de decisión o SVMs.

Las imágenes originales pasarán por cada extractor, y los _features_ y etiquetas se almacenarán en archivos `.npy`, el cual es un formato del agrado de NumPy.

### Funciones Auxiliares

Estas dos funciones nos permitirán cargar una imagen, volverla un tensor y pre-procesarla de la manera que VGG16 espera.

In [6]:
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image

def path_to_tensor(image_path):
    img = image.load_img(image_path, target_size=(224, 224))
    x = image.img_to_array(img)
    
    return np.expand_dims(x, axis=0)

def paths_to_tensor(image_paths):
    tensors = [path_to_tensor(image_path) for image_path in image_paths]
    return np.vstack(tensors)

Antes de transformar todos los datos, asegurémonos de que las cosas funcionan como deberían. Para ello, usaremos sólo unas cuantas imágenes.

In [7]:
SAMPLE_SIZE = 3

image_paths = glob('dataset/train/*/*')
image_paths = random.sample(image_paths, SAMPLE_SIZE)

# Calculate the image input
image_input = preprocess_input(paths_to_tensor(image_paths))

print(image_input.shape)

(3, 224, 224, 3)


Esas dimensiones de arriba corresponden al lote que pasaremos a cada extractor.

In [8]:
global_max_bottlenecked_batch = global_max_feature_extractor.predict(image_input)
print(f'Bottlenecked batch shape (Global Max): {global_max_bottlenecked_batch.shape}')

global_average_bottlenecked_batch = global_average_feature_extractor.predict(image_input)
print(f'Bottlenecked batch shape (Global Average): {global_average_bottlenecked_batch.shape}')

flatten_bottlenecked_batch = flatten_feature_extractor.predict(image_input)
print(f'Bottlenecked batch shape (Flatten): {flatten_bottlenecked_batch.shape}')

Bottlenecked batch shape (Global Max): (3, 512)
Bottlenecked batch shape (Global Average): (3, 512)
Bottlenecked batch shape (Flatten): (3, 25088)


Podemos apreciar que el Flatten Extractor, como esperábamos, es el que produce la representación más larga de las imánges de entrada, puesto que su operación no resume la información contenida en ellas, a diferencia de los otros dos.

### Generando las Tres Versiones de los Datos

¡Estamos listos! ¡Generemos las transformaciones!

In [9]:
from pathlib import Path

if not Path('features.npy').is_file():
    vehicles_images_path = glob('data/vehicles/*/*.png')
    non_vehicles_images_path = glob('data/non-vehicles/*/*.png')

    vehicles_images = preprocess_input(paths_to_tensor(vehicles_images_path))
    non_vehicles_images = preprocess_input(paths_to_tensor(non_vehicles_images_path))

    features = np.vstack([vehicles_images, non_vehicles_images])

    np.save('features.npy', features)
else:
    features = np.load('features.npy')
    
print(f'Features shape: {features.shape}')

Features shape: (7325, 224, 224, 3)


In [10]:
if not Path('labels.npy').is_file():
    vehicles_labels = np.array([1] * len(vehicles_images_path))
    non_vehicles_labels = np.array([0] * len(non_vehicles_images_path))
    labels = np.hstack([vehicles_labels, non_vehicles_labels])
    
    np.save('labels.npy', labels)
else:
    labels = np.load('labels.npy')

    print(f'Labels shape: {labels.shape}')

Labels shape: (7325,)


In [11]:
if not Path('global_average_features.npy').is_file():
    global_average_bottlenecked_features = global_average_feature_extractor.predict(features)
    
    np.save('global_average_features.npy', global_average_bottlenecked_features)
else:
    global_average_bottlenecked_features = np.load('global_average_features.npy')

print(f'global_average_features shape: {global_average_bottlenecked_features.shape}')

global_average_features shape: (7325, 512)


In [12]:
if not Path('global_max_features.npy').is_file():
    global_max_bottlenecked_features = global_max_feature_extractor.predict(features)
    np.save('global_max_features.npy', global_max_bottlenecked_features)
else:
    global_max_bottlenecked_features = np.load('global_max_features.npy')

print(f'global_max_bottlenecked_features shape: {global_max_bottlenecked_features.shape}')

global_max_bottlenecked_features shape: (7325, 512)


In [13]:
if not Path('flattened_features.npy').is_file():
    flattened_bottlenecked_features = flatten_feature_extractor.predict(features)
    np.save('flattened_features.npy', flattened_bottlenecked_features)
else:
    flattened_bottlenecked_features = np.load('flattened_features.npy')

print(f'flattened_bottlenecked_features shape: {flattened_bottlenecked_features.shape}')

flattened_bottlenecked_features shape: (7325, 25088)


Con estas versiones de los datos podemos ahora pasar al proceso de evaluar diferentes algoritmos con cada una de ellas.